In [1]:
# https://stackoverflow.com/questions/33508377/read-cell-content-in-an-ipython-notebook

'''
from IPython.core.magic import  (
    Magics, magics_class, cell_magic, line_magic
)
'''

In [2]:
from IPython.core.magic import Magics, magics_class, cell_magic, line_magic

@magics_class
class StoreSQL(Magics):

    def __init__(self, shell=None,  **kwargs):
        super().__init__(shell=shell, **kwargs)
        self._store = []
        # inject our store in user availlable namespace under __mystore
        # name
        shell.user_ns['__mystore'] = self._store

    @cell_magic
    def sql(self, line, cell):
        """store the cell in the store"""
        print(type(cell))
        self._store.append(cell)

    @line_magic
    def showsql(self, line):
        """show all recorded statements"""
        print(self._store)
        
## use ipython load_ext mechanisme here if distributed
get_ipython().register_magics(StoreSQL)

In [3]:
%%sql
select * from foo Where QUX Bar

<class 'str'>


In [4]:
%showsql

['select * from foo Where QUX Bar']


In [5]:
# http://ipython.readthedocs.io/en/stable/config/custommagics.html#defining-magics

from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)

@register_line_magic
def lmagic(line):
    "my line magic"
    return line

@register_cell_magic
def cmagic(line, cell):
    "my cell magic"
    return line, cell

@register_line_cell_magic
def lcmagic(line, cell=None):
    "Magic that works both as %lcmagic and as %%lcmagic"
    if cell is None:
        print("Called as line magic")
        return line
    else:
        print("Called as cell magic")
        return line, cell

# In an interactive session, we need to delete these to avoid
# name conflicts for automagic to work on line magics.
del lmagic, lcmagic

In [6]:
# https://github.com/brazilbean/modulemagic/blob/master/modulemagic/modulemagic.py
# but i don't want to save cell to a file and import again, looks silly.

In [7]:
# https://stackoverflow.com/questions/2931950/dynamic-module-creation
# https://docs.python.org/3/library/functions.html#exec
# https://docs.python.org/2.7/library/imp.html#imp.new_module

import os
import sys
import imp
import argparse

from IPython.core.magic import Magics, magics_class, cell_magic

@magics_class
class ModuleMagics(Magics):
    
    def __init__(self, shell=None, namespace=None, **kw):
        if shell is None:
            shell = get_ipython()            
        super(ModuleMagics, self).__init__(shell, **kw)
        self.namespace = namespace
        
    @cell_magic
    def module(self, line, cell):
        parser = argparse.ArgumentParser('%%module cell magic')
        parser.add_argument('fullname')
        args = parser.parse_args(line.split())

        fullname = args.fullname

        mod = sys.modules.get(fullname, imp.new_module(fullname))
        exec(cell, mod.__dict__)
        
        sys.modules[fullname] = mod
        if self.namespace:
            self.namespace[fullname] = mod
        else:
            self.shell.push({fullname: mod})

get_ipython().register_magics(ModuleMagics)

In [8]:
%%module mymod

def func():
    print("I am a function")

In [9]:
print('mymod' in sys.modules.keys())
dir(mymod)

True


['__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'func']

In [10]:
%%module mymod

def func2():
    print("I am the second function")

In [11]:
dir(mymod)
mymod.func2()

I am the second function
